In [ ]:
#Imports
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import requests
import streamlit as st
import urllib
import numpy as np

df_vereinsdaten = pd.read_csv("C:\\Users\\bened\\OneDrive - informatik.hs-fulda.de\Wintersemester 2025 2026\\05 Frameworks and Application Development for Data Science\\vereinsdaten.csv")

In [2]:
df_vereinsdaten

,scraped_club_name,strasse,plz,ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage
0,Rasdorfer Sport-Club e.V.,NaN,36169,Rasdorf,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
1,SG Alemannia 1921 Kleinlüder e.V.,An der Lüder 2,36137,Kleinlüder,http://www.sga-kleinlueder.de,Nico Gärtner,0175-1613133,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
2,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,NaN,36119,Neuhof,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
3,Sportverein 1910 Neuhof e.V.,NaN,36119,Neuhof,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
4,Sportverein Hellas 1921 Michelsrombach e.V.,Finkenweg 13,36088,Hünfeld,http://www.hellas-michelsrombach.de,Daniel Kühn,+49151 54671340,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
5,SV Buchonia 1912 Flieden 1912,Hauptstr. 19,36103,Flieden,http://www.svflieden.de,Winfried Happ,06655 / 965218; 0176 / 10052454,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
6,Tanner-Tennis-Club e.V.,Ensbach 1,36142,Tann,NaN,Jürgen Sauermilch,+49171 530535329,NaN,"Jugendliche, Erwachsene",https://www.sportkreis-fulda-huenfeld.de/verei...
7,TC 1993 Künzell Dicker Turm e.V.,Friedenstr. 45,36093,Künzell,http://www.tc-kuenzell-dicker-turm.de,TC 1993 Künzell Dicker Turm e.V.,+496619336986,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
8,TC Schwarz-Weiss 1990 Poppenhausen e.V.,NaN,36163,Poppenhausen,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
9,TCB Johannisau Fulda,Johannisstr. 47,36041,Fulda,https://www.tcb-johannisau-fulda.de/,Vorstand,+49661 71314,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...


In [3]:
df_vereine = df_vereinsdaten
club_names = df_vereine['scraped_club_name'].tolist()

In [ ]:
#Manually looking-up and adding the detail urls for htv-nuliga
df_vereine["detail_url"] = np.nan
df_vereine.loc[0, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199"
df_vereine.loc[1, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25208"
df_vereine.loc[2, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25180"
df_vereine.loc[3, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25235"
df_vereine.loc[4, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25194"
df_vereine.loc[5, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25203"
df_vereine.loc[6, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25202"
df_vereine.loc[7, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24793"
df_vereine.loc[8, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25195"
df_vereine.loc[9, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25231"
df_vereine.loc[10, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24836"
df_vereine.loc[11, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24794"
df_vereine.loc[12, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25211"
df_vereine.loc[13, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25200"
df_vereine.loc[14, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25214"
df_vereine.loc[15, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25213"
df_vereine.loc[16, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25217"
df_vereine.loc[17, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25193"
df_vereine.loc[18, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25215"
df_vereine.loc[19, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25197"
df_vereine.loc[20, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25232"
df_vereine.loc[21, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25198"
df_vereine.loc[22, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25209"
df_vereine.loc[23, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25234"
df_vereine.loc[24, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25224"
df_vereine.loc[25, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25225"
df_vereine.loc[26, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25196"
df_vereine.loc[27, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25216"
df_vereine.loc[28, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25223"
df_vereine.loc[29, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25218"
df_vereine.loc[30, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25192"

C:\Users\bened\AppData\Local\Temp\ipykernel_9096\1163436480.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_vereine.loc[0, "detail_url"] = "https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199"


In [16]:
df_vereine

,scraped_club_name,strasse,plz,ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage,detail_url
0,Rasdorfer Sport-Club e.V.,NaN,36169,Rasdorf,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
1,SG Alemannia 1921 Kleinlüder e.V.,An der Lüder 2,36137,Kleinlüder,http://www.sga-kleinlueder.de,Nico Gärtner,0175-1613133,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
2,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,NaN,36119,Neuhof,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
3,Sportverein 1910 Neuhof e.V.,NaN,36119,Neuhof,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
4,Sportverein Hellas 1921 Michelsrombach e.V.,Finkenweg 13,36088,Hünfeld,http://www.hellas-michelsrombach.de,Daniel Kühn,+49151 54671340,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
5,SV Buchonia 1912 Flieden 1912,Hauptstr. 19,36103,Flieden,http://www.svflieden.de,Winfried Happ,06655 / 965218; 0176 / 10052454,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
6,Tanner-Tennis-Club e.V.,Ensbach 1,36142,Tann,NaN,Jürgen Sauermilch,+49171 530535329,NaN,"Jugendliche, Erwachsene",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
7,TC 1993 Künzell Dicker Turm e.V.,Friedenstr. 45,36093,Künzell,http://www.tc-kuenzell-dicker-turm.de,TC 1993 Künzell Dicker Turm e.V.,+496619336986,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
8,TC Schwarz-Weiss 1990 Poppenhausen e.V.,NaN,36163,Poppenhausen,NaN,NaN,NaN,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...
9,TCB Johannisau Fulda,Johannisstr. 47,36041,Fulda,https://www.tcb-johannisau-fulda.de/,Vorstand,+49661 71314,NaN,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaT...


In [17]:
df_vereinsdaten = df_vereine

In [ ]:
def scrape_detail_page(url):
    #Defining all necessary columns before starting
    data = {
        'Platzadresse': None, 'Hallenadresse': None, 'Anlagenadresse': None,
        'Mitglieder_Maennlich_Gesamt': None, 
        'Mitglieder_Weiblich_Gesamt': None, 
        'Mitglieder_Gesamt_Gesamt': None,
        'Anzahl_Freiplaetze': None, 'Belag_Freiplaetze': None,
        'Anzahl_Hallenplaetze': None, 'Belag_Hallenplaetze': None,
        'Funktionäre': []
    }

    if pd.isna(url) or url == "":
        return data

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        response.encoding = response.apparent_encoding 
        soup = BeautifulSoup(response.text, 'html.parser')

        #Adresses (content-col1)
        col1 = soup.find('div', id='content-col1')
        if col1:
            for label in ["Platzadresse", "Hallenadresse", "Anlagenadresse"]:
                tag = col1.find('b', string=lambda s: s and label in s)
                if tag:
                    val_td = tag.find_parent('td').find_next_sibling('td')
                    if val_td:
                        for br in val_td.find_all("br"): br.replace_with(" ")
                        data[label] = val_td.get_text(strip=True)

        #Members & Courts (content-col2)
        col2 = soup.find('div', id='content-col2')
        if col2:
         #Searching in the t-foot part of the table
            m_table = col2.find('table', class_='stocktaking-members')
            if m_table:
                    tfoot = m_table.find('tfoot')
                    if tfoot:
                        tfoot_cells = tfoot.find_all('td')
                        # Index 1: Männlich Gesamt, Index 2: Weiblich Gesamt, Index 3: Gesamt Gesamt
                        if len(tfoot_cells) >= 4:
                            data['Mitglieder_Maennlich_Gesamt'] = tfoot_cells[1].get_text(strip=True)
                            data['Mitglieder_Weiblich_Gesamt'] = tfoot_cells[2].get_text(strip=True)
                            data['Mitglieder_Gesamt_Gesamt'] = tfoot_cells[3].get_text(strip=True)

#Extracting courts (Indoor & Outdoor)
            h2_p = col2.find('h2', string=lambda s: s and "Plätze" in s)
            if h2_p:
                p_table = h2_p.find_next('table')
                for p_row in p_table.find_all('tr'):
                    p_cells = p_row.find_all('td')
                    if len(p_cells) >= 3:
                        anzahl = p_cells[0].get_text(strip=True).replace('\xa0', '')
                        art = p_cells[1].get_text(strip=True)
                        belag = p_cells[2].get_text(strip=True)
                        
                        #Dynamic assigning based on the court type
                        if "Freiplätze" in art:
                            data['Anzahl_Freiplaetze'] = anzahl
                            data['Belag_Freiplaetze'] = belag
                        elif "Hallenplätze" in art:
                            data['Anzahl_Hallenplaetze'] = anzahl
                            data['Belag_Hallenplaetze'] = belag

        #Funktionäre
        row2 = soup.find('div', id='content-row2')
        if row2:
            f_table = row2.find('table', class_='result-set')
            if f_table:
                #Starting from line 3
                for r in f_table.find_all('tr')[2:]:
                    c = r.find_all('td')
                    if len(c) >= 2:
                        data['Funktionäre'].append({
                            "Funktion": c[0].get_text(strip=True),
                            "Name": c[1].get_text(strip=True),
                            "Telefon": c[2].get_text(strip=True) if len(c) > 2 else "",
                            "Email": c[3].get_text(strip=True) if len(c) > 3 else ""
                        })
        return data
    except Exception as e:
        print(f"Fehler bei {url}: {e}")
        return data

In [ ]:
df_total = df_vereine.copy() 

scraped_results = []
total_rows = len(df_total)

for idx, row in df_total.iterrows():
    # Progress tracking for all rows
    print(f"Scrape {idx+1}/{total_rows}: {row['scraped_club_name']}")
    
    details = scrape_detail_page(row['detail_url'])
    scraped_results.append({**row.to_dict(), **details})
    
    #Time.sleep(1) for not getting blocked
    time.sleep(1)

df_final = pd.DataFrame(scraped_results)

# Exploding Funktionäre & splitting columns
df_final = df_final.explode('Funktionäre').reset_index(drop=True)
f_details = pd.json_normalize(df_final['Funktionäre'])
df_final = pd.concat([df_final.drop(columns=['Funktionäre']), f_details], axis=1)

Scrape 1/31: Rasdorfer Sport-Club e.V.
Scrape 2/31: SG Alemannia 1921 Kleinlüder e.V.
Scrape 3/31: SG Schwarz-Weiß 1919/45 Hattenhof e.V.
Scrape 4/31: Sportverein 1910 Neuhof e.V.
Scrape 5/31: Sportverein Hellas 1921 Michelsrombach e.V.
Scrape 6/31: SV Buchonia 1912 Flieden 1912
Scrape 7/31: Tanner-Tennis-Club e.V.
Scrape 8/31: TC 1993 Künzell Dicker Turm e.V.
Scrape 9/31: TC Schwarz-Weiss 1990 Poppenhausen e.V.
Scrape 10/31: TCB Johannisau Fulda
Scrape 11/31: Tennis Club 88 Mackenzell
Scrape 12/31: Tennis Club Blau-Weiß Hilders e.V.
Scrape 13/31: Tennis Club Nüsttal e.V.
Scrape 14/31: Tennis-Club 1993 Altenhof
Scrape 15/31: Tennis-Club Grün-Weiß e.V. Fulda
Scrape 16/31: Tennisclub 1980 Eichenzell e.V.
Scrape 17/31: Tennisclub 1987 Marbach e.V.
Scrape 18/31: Tennisclub 1990 Hofbieber
Scrape 19/31: Tennisclub 77 Ebersburg/Rhön e.V.
Scrape 20/31: Tennisclub Blau-Weiß Hünfeld 1951 e.V.
Scrape 21/31: Tennisclub Blau-Weiß Petersberg e.V.
Scrape 22/31: Tennisclub Burghaun e.V.
Scrape 23/31: 

In [35]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
#Cleaning and Extracting address data
def extract_and_clean_data(row):
    address_cols = ['Platzadresse', 'Hallenadresse', 'Anlagenadresse']
    
    url_pattern = r'(https?://[^\s]+|www\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
   
    tel_pattern = r'(?:Mobil|Tel\s+[PG])\s*[:\s]*(\+?[0-9\-\s/]{8,})|(\+?[0-9]{10,})'

    for col in address_cols:
        text = row[col]
        if not text or not isinstance(text, str) or "keine Adressinformationen" in text:
            continue
            
        #Using website information hidden in address data
        if pd.isna(row['website']) or row['website'] == '':
            found_urls = re.findall(url_pattern, text)
            if found_urls:
                row['website'] = found_urls[0]

        #Using email information hidden in address data
        if pd.isna(row['email']) or row['email'] == '':
            found_emails = re.findall(email_pattern, text)
            if found_emails:
                row['email'] = found_emails[0]

        #Using telephone numbers hidden in address data
        if pd.isna(row['telefon']) or row['telefon'] == '':
            found_tels = re.search(tel_pattern, text)
            if found_tels:
                tel_val = found_tels.group(1) if found_tels.group(1) else found_tels.group(2)
                row['telefon'] = tel_val.strip()

        #Cleaning address data
        text = re.sub(url_pattern, '', text)
        text = re.sub(email_pattern, '', text)
    
        text = re.sub(r'(Mobil|Tel\s+[PG])\s*[:\s]*\+?[0-9\-\s/]+', '', text)
       
        text = re.sub(r'\d{10,}', '', text)
        
        text = text.replace("Deutschlandwww", "Deutschland")
        text = text.replace("DeutschlandMobil", "Deutschland")
        
        row[col] = re.sub(r'\s+', ' ', text).strip()

    return row

In [37]:
df_final = df_final.apply(extract_and_clean_data, axis=1)

In [ ]:
#Manually correcting last problems with addresses
df_final.loc[[0, 1, 2, 3, 4, 5, 6, 7], 'Platzadresse'] = 'Eckbertstraße 6a, 36169 Rasdorf, Deutschland'
df_final.loc[[0, 1, 2, 3, 4, 5, 6, 7], 'Anlagenadresse'] = 'Eckbertstraße 6a, 36169 Rasdorf, Deutschland'
df_final.loc[[22, 23, 24, 25, 26, 27, 28, 29, 30], 'Platzadresse'] = 'Hanauer Straße 14, 36119 Neuhof, Deutschland'
df_final.loc[[36, 37, 38, 39, 40, 41, 42, 43, 44, 45], 'Platzadresse'] = 'Am Mühlbach, 36103 Flieden, Deutschland'
df_final.loc[[36, 37, 38, 39, 40, 41, 42, 43, 44, 45], 'Anlagenadresse'] = 'Am Mühlbach, 36103 Flieden, Deutschland'
df_final.loc[[53, 54, 55, 56, 57, 58, 59], 'Platzadresse'] = 'Friedenstraße 45, 36093 Künzell, Deutschland'
df_final.loc[[53, 54, 55, 56, 57, 58, 59], 'Anlagenadresse'] = 'Friedenstraße 45, 36093 Künzell, Deutschland'
df_final.loc[[100, 101, 102, 103, 104, 105], 'Platzadresse'] = 'Am Sportplatz 1, 97786 Motten, Deutschland'
df_final.loc[[100, 101, 102, 103, 104, 105], 'Anlagenadresse'] = 'Am Sportplatz 1, 97786 Motten, Deutschland'
df_final.loc[[106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118], 'Platzadresse'] = 'Olympiastraße 10, 36041 Fulda, Deutschland'
df_final.loc[[106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118], 'Hallenadresse'] = 'Olympiastraße 10, 36041 Fulda, Deutschland'
df_final.loc[[106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118], 'Anlagenadresse'] = 'Olympiastraße 10, 36041 Fulda, Deutschland'
df_final.loc[[130, 131, 132, 133, 134, 135, 136, 137, 138], 'Platzadresse'] = 'Im Ehrlich 1, 36100 Petersberg, Deutschland'
df_final.loc[[130, 131, 132, 133, 134, 135, 136, 137, 138], 'Anlagenadresse'] = 'Im Ehrlich 1, 36100 Petersberg, Deutschland'
df_final.loc[[164, 165, 166, 167, 168], 'Platzadresse']  = 'Am Pfaffenpfad 9, 36100 Petersberg, Deutschland'
df_final.loc[[170, 171, 172, 173, 174, 175, 176, 177], 'Platzadresse']  = 'Wendelinusstraße 50, 36151 Burghaun, Deutschland'
df_final.loc[[178, 179, 180, 181, 182, 183, 184], 'Platzadresse']  = 'Lindenweg 2, 36160 Dipperz, Deutschland'
df_final.loc[[199, 200, 201, 202, 203, 204], 'Platzadresse'] = 'Marienstrasse 18, 36039 Fulda, Deutschland'
df_final.loc[[230, 231, 232, 233, 234, 235, 236], 'Platzadresse'] = 'Am Schmidtwasser, 36148 Kalbach, Deutschland'
df_final.loc[[230, 231, 232, 233, 234, 235, 236], 'Anlagenadresse'] = 'Am Schmidtwasser, 36148 Kalbach, Deutschland'

In [71]:
df_final.drop(columns="Belag_Hallenplaetze")

,scraped_club_name,strasse,plz,ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage,detail_url,Platzadresse,Hallenadresse,Anlagenadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Funktion,Name,Telefon,Email
0,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Vorsitzende:r/Abteilungsleiter:in,"Seidel, Adrian",0160 90 28 28 73,
1,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Sportwart:in,"von Lorentz, Anna",017623171091,
2,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Jugendwart:in,"Fischer, Anna",,-
3,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Jugendwart:in,"Bohn, Ursula",06651 - 797,-
4,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Breitensportwart:in,"von Lorentz, Anna",017623171091,
5,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,Kassenwart:in,"Rollmann, Sabrina",0171 2169692,
6,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuL

In [61]:
df_final.columns.tolist()

['scraped_club_name',
 'strasse',
 'plz',
 'ort',
 'website',
 'ansprechpartner',
 'telefon',
 'email',
 'zielgruppe',
 'scraped_link_detailpage',
 'detail_url',
 'Platzadresse',
 'Hallenadresse',
 'Anlagenadresse',
 'Mitglieder_Maennlich_Gesamt',
 'Mitglieder_Weiblich_Gesamt',
 'Mitglieder_Gesamt_Gesamt',
 'Anzahl_Freiplaetze',
 'Belag_Freiplaetze',
 'Anzahl_Hallenplaetze',
 'Belag_Hallenplaetze',
 'Funktion',
 'Name',
 'Telefon',
 'Email']

In [85]:
df_unique = df_final.drop_duplicates(subset=['scraped_club_name']).copy()

spalten_zu_entfernen = ['strasse','plz','ort','ansprechpartner','telefon','email','zielgruppe','scraped_link_detailpage','Hallenadresse','Anlagenadresse',
                        'Funktion','Name','Telefon','Email'] 

df_unique = df_unique.drop(columns=[col for col in spalten_zu_entfernen if col in df_unique.columns])

In [86]:
df_unique

,scraped_club_name,website,detail_url,Platzadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Belag_Hallenplaetze
0,Rasdorfer Sport-Club e.V.,www.tennis-rasdorf.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-
8,SG Alemannia 1921 Kleinlüder e.V.,http://www.sga-kleinlueder.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25208,"Am Reet, 36137 Großenlüder, Deutschland",31,2,33,2,Sand,0,-
14,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25180,"Am Rippberg, 36119 Neuhof, Deutschland",56,6,62,2,Quarzsand,0,-
22,Sportverein 1910 Neuhof e.V.,www.tennis-neuhof.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25235,"Hanauer Straße 14, 36119 Neuhof, Deutschland",61,52,113,4,Sand,0,-
31,Sportverein Hellas 1921 Michelsrombach e.V.,http://www.hellas-michelsrombach.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25194,"Koenigskueppel, 36088 Huenfeld, Deutschland",46,55,101,2,Quarzsand,0,-
36,SV Buchonia 1912 Flieden 1912,http://www.svflieden.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25203,"Am Mühlbach, 36103 Flieden, Deutschland",59,33,92,3,Sand,0,-
46,Tanner-Tennis-Club e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25202,"Bahnhofstr., 36142 Tann, Deutschland,",33,20,53,3,Quarzsand,0,-
53,TC 1993 Künzell Dicker Turm e.V.,http://www.tc-kuenzell-dicker-turm.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24793,"Friedenstraße 45, 36093 Künzell, Deutschland",118,81,199,4,Aschenplatz,0,-
60,TC Schwarz-Weiss 1990 Poppenhausen e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25195,"Luettergrund, 36163 Poppenhausen, Deutschland",56,33,89,2,Quarzsand,0,-
67,TCB Johannisau Fulda,https://www.tcb-johannisau-fulda.de/,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25231,"Johannisstraße 47, 36041 Fulda, Deutschland",89,93,182,4,Sand,0,-


In [ ]:
#Geocoding coordinates from addresses for the interactive map
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

#Initialising Geocoder
geolocator = Nominatim(user_agent="tennis_club_locator_project")

#Configurating RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

#Geocoding and creating a new column location
print("Starte Geocoding... Bitte warten (ca. 1 Sekunde pro Verein).")
df_unique['location'] = df_unique['Platzadresse'].apply(geocode)

#Extracting latitude and longitude from location into two new separate columns
df_unique['latitude'] = df_unique['location'].apply(lambda loc: loc.latitude if loc else None)
df_unique['longitude'] = df_unique['location'].apply(lambda loc: loc.longitude if loc else None)

missing = df_unique['latitude'].isna().sum()
print(f"Geocoding beendet. {len(df_unique) - missing} Vereine gefunden, {missing} Fehler.")

#Removing column location
df_unique_final = df_unique.drop(columns=['location'])

Starte Geocoding... Bitte warten (ca. 1 Sekunde pro Verein).
Geocoding beendet. 30 Vereine gefunden, 1 Fehler.


In [88]:
df_unique_final

,scraped_club_name,website,detail_url,Platzadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Belag_Hallenplaetze,latitude,longitude
0,Rasdorfer Sport-Club e.V.,www.tennis-rasdorf.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,50.715729,9.902990
8,SG Alemannia 1921 Kleinlüder e.V.,http://www.sga-kleinlueder.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25208,"Am Reet, 36137 Großenlüder, Deutschland",31,2,33,2,Sand,0,-,50.546093,9.510852
14,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25180,"Am Rippberg, 36119 Neuhof, Deutschland",56,6,62,2,Quarzsand,0,-,50.464822,9.681020
22,Sportverein 1910 Neuhof e.V.,www.tennis-neuhof.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25235,"Hanauer Straße 14, 36119 Neuhof, Deutschland",61,52,113,4,Sand,0,-,50.451133,9.611403
31,Sportverein Hellas 1921 Michelsrombach e.V.,http://www.hellas-michelsrombach.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25194,"Koenigskueppel, 36088 Huenfeld, Deutschland",46,55,101,2,Quarzsand,0,-,50.661563,9.665803
36,SV Buchonia 1912 Flieden 1912,http://www.svflieden.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25203,"Am Mühlbach, 36103 Flieden, Deutschland",59,33,92,3,Sand,0,-,50.423606,9.562987
46,Tanner-Tennis-Club e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25202,"Bahnhofstr., 36142 Tann, Deutschland,",33,20,53,3,Quarzsand,0,-,50.647755,10.020986
53,TC 1993 Künzell Dicker Turm e.V.,http://www.tc-kuenzell-dicker-turm.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24793,"Friedenstraße 45, 36093 Künzell, Deutschland",118,81,199,4,Aschenplatz,0,-,50.534681,9.732342
60,TC Schwarz-Weiss 1990 Poppenhausen e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25195,"Luettergrund, 36163 Poppenhausen, Deutschland",56,33,89,2,Quarzsand,0,-,50.490019,9.876785
67,TCB Johannisau Fulda,https://www.tcb-johannisau-fulda.de/,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25231,"Johannisstraße 47, 36041 Fulda, Deutschland",89,93,182,4,Sand,0,-,50.541258,9.666585


In [ ]:
#Manual lookup of coordinates for Tennisclub Uttrichshausen 82 Kalbach Rhön e.V
df_unique_final.loc[df_unique_final['scraped_club_name'] == 'Tennisclub Uttrichshausen 82 Kalbach Rhön e.V.', 'latitude'] = 50.41179
df_unique_final.loc[df_unique_final['scraped_club_name'] == 'Tennisclub Uttrichshausen 82 Kalbach Rhön e.V.', 'longitude'] = 9.73434

In [90]:
df_unique_final

,scraped_club_name,website,detail_url,Platzadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Belag_Hallenplaetze,latitude,longitude
0,Rasdorfer Sport-Club e.V.,www.tennis-rasdorf.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,50.715729,9.902990
8,SG Alemannia 1921 Kleinlüder e.V.,http://www.sga-kleinlueder.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25208,"Am Reet, 36137 Großenlüder, Deutschland",31,2,33,2,Sand,0,-,50.546093,9.510852
14,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25180,"Am Rippberg, 36119 Neuhof, Deutschland",56,6,62,2,Quarzsand,0,-,50.464822,9.681020
22,Sportverein 1910 Neuhof e.V.,www.tennis-neuhof.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25235,"Hanauer Straße 14, 36119 Neuhof, Deutschland",61,52,113,4,Sand,0,-,50.451133,9.611403
31,Sportverein Hellas 1921 Michelsrombach e.V.,http://www.hellas-michelsrombach.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25194,"Koenigskueppel, 36088 Huenfeld, Deutschland",46,55,101,2,Quarzsand,0,-,50.661563,9.665803
36,SV Buchonia 1912 Flieden 1912,http://www.svflieden.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25203,"Am Mühlbach, 36103 Flieden, Deutschland",59,33,92,3,Sand,0,-,50.423606,9.562987
46,Tanner-Tennis-Club e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25202,"Bahnhofstr., 36142 Tann, Deutschland,",33,20,53,3,Quarzsand,0,-,50.647755,10.020986
53,TC 1993 Künzell Dicker Turm e.V.,http://www.tc-kuenzell-dicker-turm.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24793,"Friedenstraße 45, 36093 Künzell, Deutschland",118,81,199,4,Aschenplatz,0,-,50.534681,9.732342
60,TC Schwarz-Weiss 1990 Poppenhausen e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25195,"Luettergrund, 36163 Poppenhausen, Deutschland",56,33,89,2,Quarzsand,0,-,50.490019,9.876785
67,TCB Johannisau Fulda,https://www.tcb-johannisau-fulda.de/,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25231,"Johannisstraße 47, 36041 Fulda, Deutschland",89,93,182,4,Sand,0,-,50.541258,9.666585


In [91]:
text_spalten = ['website', 'telefon', 'email', 'Funktion', 'Name', 'Platzadresse', 'Hallenadresse','Anlagenadresse']
df_final[text_spalten] = df_final[text_spalten].fillna("unbekannt")

In [92]:
df_final.to_csv("df_clubs_complete_final.csv", index=False, encoding="utf-8")
df_unique_final.to_csv("df_clubs_unique_final.csv", index=False, encoding="utf-8")

In [93]:
df_unique_final

,scraped_club_name,website,detail_url,Platzadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Belag_Hallenplaetze,latitude,longitude
0,Rasdorfer Sport-Club e.V.,www.tennis-rasdorf.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,50.715729,9.902990
8,SG Alemannia 1921 Kleinlüder e.V.,http://www.sga-kleinlueder.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25208,"Am Reet, 36137 Großenlüder, Deutschland",31,2,33,2,Sand,0,-,50.546093,9.510852
14,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25180,"Am Rippberg, 36119 Neuhof, Deutschland",56,6,62,2,Quarzsand,0,-,50.464822,9.681020
22,Sportverein 1910 Neuhof e.V.,www.tennis-neuhof.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25235,"Hanauer Straße 14, 36119 Neuhof, Deutschland",61,52,113,4,Sand,0,-,50.451133,9.611403
31,Sportverein Hellas 1921 Michelsrombach e.V.,http://www.hellas-michelsrombach.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25194,"Koenigskueppel, 36088 Huenfeld, Deutschland",46,55,101,2,Quarzsand,0,-,50.661563,9.665803
36,SV Buchonia 1912 Flieden 1912,http://www.svflieden.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25203,"Am Mühlbach, 36103 Flieden, Deutschland",59,33,92,3,Sand,0,-,50.423606,9.562987
46,Tanner-Tennis-Club e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25202,"Bahnhofstr., 36142 Tann, Deutschland,",33,20,53,3,Quarzsand,0,-,50.647755,10.020986
53,TC 1993 Künzell Dicker Turm e.V.,http://www.tc-kuenzell-dicker-turm.de,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=24793,"Friedenstraße 45, 36093 Künzell, Deutschland",118,81,199,4,Aschenplatz,0,-,50.534681,9.732342
60,TC Schwarz-Weiss 1990 Poppenhausen e.V.,unbekannt,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25195,"Luettergrund, 36163 Poppenhausen, Deutschland",56,33,89,2,Quarzsand,0,-,50.490019,9.876785
67,TCB Johannisau Fulda,https://www.tcb-johannisau-fulda.de/,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25231,"Johannisstraße 47, 36041 Fulda, Deutschland",89,93,182,4,Sand,0,-,50.541258,9.666585


In [94]:
df_final

,scraped_club_name,strasse,plz,ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage,detail_url,Platzadresse,Hallenadresse,Anlagenadresse,Mitglieder_Maennlich_Gesamt,Mitglieder_Weiblich_Gesamt,Mitglieder_Gesamt_Gesamt,Anzahl_Freiplaetze,Belag_Freiplaetze,Anzahl_Hallenplaetze,Belag_Hallenplaetze,Funktion,Name,Telefon,Email
0,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Vorsitzende:r/Abteilungsleiter:in,"Seidel, Adrian",0160 90 28 28 73,
1,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Sportwart:in,"von Lorentz, Anna",017623171091,
2,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Jugendwart:in,"Fischer, Anna",,-
3,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Jugendwart:in,"Bohn, Ursula",06651 - 797,-
4,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Breitensportwart:in,"von Lorentz, Anna",017623171091,
5,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://htv.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubInfoDisplay?club=25199,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",Es sind keine Adressinformationen erfasst.,"Eckbertstraße 6a, 36169 Rasdorf, Deutschland",44,50,94,2,Sand,0,-,Kassenwart:in,"Rollmann, Sabrina",0171 2169692,
6,Rasdorfer Sport-Club e.V.,unbekannt,36169,Rasdorf,www.tennis-rasdorf.de,unbekannt,unbekannt,unbekannt,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene, Senioren",https://www.sportkreis-fulda-huenfeld.de/vereine/ra32xshu_rasdorfer_sport-club_ev.html?sa=449&formstep=suche,https://ht